In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings

/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings


In [ ]:
# download dataset
import requests
file_url = "http://205.174.165.80/CICDataset/ISCX-URL-2016/Dataset/ISCXURL2016.zip"
r = requests.get(file_url, stream = True)
with open("/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/ISCXURL2016.zip","wb") as file:
  for block in r.iter_content(chunk_size = 1024):
    if block:
      file.write(block)

In [ ]:
!ls

FinalDataset  ISCXURL2016.zip  Nhan_Extracting.ipynb


In [ ]:
!unzip ISCXURL2016.zip

unzip:  cannot find or open ISCXURL2016.zip, ISCXURL2016.zip.zip or ISCXURL2016.zip.ZIP.


In [ ]:
import pandas as pd
import numpy as np

#create dummies from TLD column
tldmat = pd.get_dummies(data.tld)

##label new dummies matrix
tldmat['tag'] = data['tag']

### subset malicious and benign sampls
tldmal = tldmat[tldmat['tag'] == 'malware']
tldben = tldmat[tldmat['tag'] == 'benign']

dd = []
for i in tldmat.columns[:-1]:
    md = np.mean(tldmal[i]) - np.mean(tldben[i])
    y = ttest_ind(tldmal[i], tldben[i])
    d2 = [i, md, y[1]]
    dd.append(d2)
    
dd = pd.DataFrame(dd)
dd.columns = ['Feature', 'Mean_Difference', 'P_Value']
dd = dd.sort_values(['Mean_Difference', 'P_Value'], ascending=[False, True])


In [ ]:
import pandas as pd
data_benign = pd.read_csv('/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/URL/Benign.csv', header=None)
data_malware = pd.read_csv('/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/URL/Malware.csv', header=None)

In [ ]:
data_benign.head(5)

,0
0,http://1337x.to/torrent/1048648/American-Snipe...
1,http://1337x.to/torrent/1110018/Blackhat-2015-...
2,http://1337x.to/torrent/1122940/Blackhat-2015-...
3,http://1337x.to/torrent/1124395/Fast-and-Furio...
4,http://1337x.to/torrent/1145504/Avengers-Age-o...


#Lexical Features

In [ ]:
!pip install pyquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from math import log
from re import compile
from urllib.parse import urlparse
from sklearn.feature_extraction.text import TfidfVectorizer
from socket import gethostbyname
from pyquery import PyQuery
from requests import get
from json import dump
from string import ascii_lowercase
from numpy import array

cache = []

with open('FinalDataset/output/lexicalfeatures.json', 'r+') as ff:
   data = [loads(i) for i in ff.readlines()]
   seen = [i['host'] for i in data]
   for i in seen:
       print('Caching {}'.format(i))
       cache.append(i)

'''Get Alexa top 50 sites'''
def __get_alexa_top_50():
    pq = PyQuery(get('https://www.alexa.com/topsites').content)
    items = pq('.site-listing .DescriptionCell p a')
    sites = [i.text().lower() for i in items.items()]
    return sites
alexa_50 = __get_alexa_top_50()


class alexa_similarity:
    def __init__(self, url):
        self.alexa50 = alexa_50
        self.url = url

    '''Remove none numeric or alphabetic characters from URL strings'''
    def clean_url(self, url):
        url = ''.join([i for i in url.lower() if i.isalpha() or i.isalpha()])
        return url

    '''Get list of common characters between two URL strings'''
    def similar_string_score(self, string1:str, string2: str):
        string1, string2 = string1.lower(), string2.lower()
        return list(set(string1) & set(string2))

    '''count the frequency occurance of common charaters in both strings'''
    def count_freq_similar(self, string1: str, string2: str):
        sims = self.similar_string_score(string1, string2)
        if len(sims) == 0:
            return 0
        sim1 = [string1.count(i) for i in sims]
        sim2 = [string2.count(i) for i in sims]
        a = sum(abs(array(sim1) - array(sim2)))
        return a

    '''Get the ascii positional index of a letter'''
    def get_pos(self, char: str):
        try:
            pos = ascii_lowercase.index(char.lower()) if char.isalpha else char
            return int(pos)
        except:
            return 0

    '''Estimate alpha numeric distribution difference'''
    def andd(self, string1: str, string2: str):
        corpus = list(map(self.clean_url, [string1, string2]))
        shorter, longer = min(corpus, key=len), max(corpus, key=len)
        diff = len(longer) - len(shorter)
        corpus[corpus.index(shorter)] = shorter + ('0' * diff)
        diffs = [abs(self.get_pos(corpus[0][i]) - self.get_pos(corpus[1][i])) for i in range(0, len(longer))]
        return sum(diffs)/len(diffs)

    '''Estimate how dis similar 2 URLs are'''
    def alexa_dis_similarity(self):
        self.url = self.clean_url(self.url)
        x = {i: self.count_freq_similar(self.url, self.clean_url(i)) for i in self.alexa50}
        x = {k: v for k, v in x.items() if v < 5}
        print(x)
        if len(x) > 0:
            diffs = [self.andd(self.url, self.clean_url(k)) for k in list(x.keys())]
            return sum(diffs)/len(diffs)
        else:
            return 0


class LexicalURLFeature:
    def __init__(self, url):
        self.description = 'blah'
        self.url = url
        self.urlparse = urlparse(self.url)
        self.alexa = alexa_similarity(self.url)
        self.host = self.__get_ip()


    def __get_entropy(self, text):
        text = text.lower()
        probs = [text.count(c) / len(text) for c in set(text)]
        entropy = -sum([p * log(p) / log(2.0) for p in probs])
        return entropy

    def __get_ip(self):
        try:
            ip = self.urlparse.netloc if self.url_host_is_ip() else gethostbyname(self.urlparse.netloc)
            return ip
        except:
            return None

    # extract lexical features
    def url_scheme(self):
        print(self.url)
        print(self.urlparse)
        return self.urlparse.scheme

    def url_length(self):
        return len(self.url)

    def url_path_length(self):
        return len(self.urlparse.path)

    def url_host_length(self):
        return len(self.urlparse.netloc)

    def url_host_is_ip(self):
        host = self.urlparse.netloc
        pattern = compile("^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$")
        match = pattern.match(host)
        return match is not None

    def url_has_port_in_string(self):
        has_port = self.urlparse.netloc.split(':')
        return len(has_port) > 1 and has_port[-1].isdigit()

    def number_of_digits(self):
        digits = [i for i in self.url if i.isdigit()]
        return len(digits)

    def number_of_parameters(self):
        params = self.urlparse.query
        return 0 if params == '' else len(params.split('&'))

    def number_of_fragments(self):
        frags = self.urlparse.fragment
        return len(frags.split('#')) - 1 if frags == '' else 0

    def is_encoded(self):
        return '%' in self.url.lower()

    def num_encoded_char(self):
        encs = [i for i in self.url if i == '%']
        return len(encs)

    def url_string_entropy(self):
        return self.__get_entropy(self.url)

    def average_alexa_50_similarity(self):
        return self.alexa.alexa_dis_similarity()

    def number_of_subdirectories(self):
        d = self.urlparse.path.split('/')
        return len(d)

    def number_of_periods(self):
        periods = [i for i in self.url if i == '.']
        return len(periods)

    def has_client_in_string(self):
        return 'client' in self.url.lower()

    def has_admin_in_string(self):
        return 'admin' in self.url.lower()

    def has_server_in_string(self):
        return 'server' in self.url.lower()

    def has_login_in_string(self):
        return 'login' in self.url.lower()

    def get_tld(self):
        return self.urlparse.netloc.split('.')[-1].split(':')[0]

    def run(self):
        try:
            fv = {
                  'host': self.host,
                   'tld': self.get_tld(),
                  'scheme': self.url_scheme(),
                  'url_length': self.url_length(),
                  'path_length': self.url_path_length(),
                  'host_length': self.url_host_length(),
                  'host_is_ip': self.url_host_is_ip(),
                  'has_port_in_string': self.url_has_port_in_string(),
                  'num_digits': self.number_of_digits(),
                  'parameters': self.number_of_parameters(),
                  'fragments': self.number_of_fragments(),
                  'is_encoded': self.is_encoded(),
                  'string_entropy': self.url_string_entropy(),
                  'alexa_dis_similarity': self.average_alexa_50_similarity(),
                  'subdirectories': self.number_of_subdirectories(),
                  'periods': self.number_of_periods(),
                  'has_client': self.has_client_in_string(),
                  'has_login': self.has_login_in_string(),
                  'has_admin': self.has_admin_in_string(),
                  'has_server': self.has_server_in_string(),
                  'num_encoded_chars': self.num_encoded_char()
                     }
            return fv
        except:
            pass
#ob = LexicalURLFeature("http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/")
#ob.run()

if __name__ == '__main__':
    # files = os.listdir('data')
    files = ['Benign.csv', 'Malware.csv']
    for file in files:
        tag, file = file.split('.')[0], 'FinalDataset/URL/{}'.format(file)
        with open(file, 'r+') as ff:
            lines = [i.strip() for i in ff.readlines()][::-1]
            for line in lines:
                ft = LexicalURLFeature(line).run()
                if ft is not None:
                    ft['tag'] = tag
                    with open('FinalDataset/output/lexicalfeatures.json', 'a') as jj:
                        dump(ft, jj)
                        jj.write('\n')
                        print(ft)
                else:
                    pass

#Content-based Features

In [ ]:
!pip install pyquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from urllib.parse import urlparse
from pyquery import PyQuery
from requests import get
from socket import gethostbyname
from numpy import array, log
from string import punctuation
from json import dump, loads
from re import compile

cache = []

with open('FinalDataset/output/contentfeatures.json', 'r+') as ff:
   data = [loads(i) for i in ff.readlines()]
   seen = [i['host'] for i in data]
   for i in seen:
       print('Caching {}'.format(i))
       cache.append(i)

def __get_valid_html_tags():
    pq = PyQuery(get('https://htmldog.com/references/html/tags/').content)
    items = pq('.longlist.acodeblock ul li a code')
    tags = [i.text().lower() for i in items.items()]
    return tags

def __get_suspicious_functions(url='https://gist.githubusercontent.com/eneyi/5c0b33129bcbfa366eb9fe79e96c1996/raw/96217aa7ea6698b17151f866f891ba701cbd7537/mal_script_functions.txt'):
    content = get(url).text.split('\n')
    return content

vd = __get_valid_html_tags()
sf = __get_suspicious_functions()

class ContentFeatures:
    def __init__(self, url, vd = vd, sf = sf):
        self.url = url
        self.urlparse = urlparse(self.url)
        self.host = self.__get_ip()
        self.html = self.__get_html()
        self.pq = self.__get_pq()
        self.scripts = self.__get_scripts()
        self.valid_tags = vd
        self.suspicious_functions = sf

    def __get_ip(self):
        try:
            ip = self.urlparse.netloc if self.url_host_is_ip() else gethostbyname(self.urlparse.netloc)
            return ip
        except:
            return None


    def url_host_is_ip(self):
        host = self.urlparse.netloc
        pattern = compile("^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$")
        match = pattern.match(host)
        return match is not None

    def __get_html(self):
        try:
            html = get(self.url, timeout=5)
            html = html.text if html else None
        except:
            html = None
        return html

    def __get_pq(self):
        try:
            pq = PyQuery(self.html) if self.html else None
            return pq
        except:
            return None


    def __get_scripts(self):
        scripts = self.pq('script') if self.pq else None
        return scripts

    def __get_entropy(self, text):
        text = text.lower()
        probs = [text.count(c) / len(text) for c in set(text)]
        return -sum([p * log(p) / log(2.0) for p in probs])

    #extract content-based features
    def url_page_entropy(self):
        return self.__get_entropy(self.html)

    def number_of_script_tags(self):
        return len(self.scripts) if self.scripts else None

    def script_to_body_ratio(self):
        if self.scripts:
            scripts = self.scripts.text()
            return len(scripts)/self.length_of_html()
        else:
            return None

    def length_of_html(self):
        return len(self.html)

    def number_of_page_tokens(self):
        html_tokens = len(self.html.lower().split()) if self.html else None
        return html_tokens

    def number_of_sentences(self):
        html_sentences = len(self.html.split('.')) if self.html else None
        return html_sentences

    def number_of_punctuations(self):
        excepts = ['<', '>', '/']
        matches = [i for i in self.html if i in punctuation and i not in excepts]
        return len(matches)

    def number_of_distinct_tokens(self):
        html_tokens = [i.strip() for i in self.html.lower().split()]
        return len(set(html_tokens))

    def number_of_capitalizations(self):
        uppercases = [i for i in self.html if i.isupper()]
        return len(uppercases)

    def average_number_of_tokens_in_sentence(self):
        html_sentences = self.html.split('.')
        sen_lens = [len(i.split()) for i in html_sentences]
        return sum(sen_lens)/len(sen_lens)

    def number_of_html_tags(self):
        return len(self.pq('*')) if self.pq else None

    def number_of_hidden_tags(self):
        hidden1, hidden2 = self.pq('.hidden'), self.pq('#hidden')
        hidden3, hidden4 = self.pq('*[visibility="none"]'), self.pq('*[display="none"]')
        hidden = hidden1 + hidden2 + hidden3 + hidden4
        return len(hidden)

    def number_iframes(self):
        iframes = self.pq('iframe') + self.pq('frame')
        return len(iframes)

    def number_objects(self):
        objects = self.pq('object')
        return len(objects)

    def number_embeds(self):
        objects = self.pq('embed')
        return len(objects)

    def number_of_hyperlinks(self):
        hyperlinks = self.pq('a')
        return len(hyperlinks)

    def number_of_whitespace(self):
        whitespaces = [i for i in self.html if i == ' ']
        return len(whitespaces)

    def number_of_included_elements(self):
        toi = self.pq('script') + self.pq('iframe') + self.pq('frame') + self.pq('embed') + self.pq('form') + self.pq('object')
        toi = [tag.attr('src') for tag in toi.items()]
        return len([i for i in toi if i])

    def number_of_suspicious_elements(self):
        all_tags = [i.tag for i in self.pq('*')]
        suspicious = [i for i in all_tags if i not in self.valid_tags]
        return len(suspicious)

    def number_of_double_documents(self):
        tags = self.pq('html') + self.pq('body') + self.pq('title')
        return len(tags) - 3

    def number_of_eval_functions(self):
        scripts = self.pq('script')
        scripts = ['eval' in script.text().lower() for script in scripts.items()]
        return sum(scripts)

    def average_script_length(self):
        scripts = self.pq('script')
        scripts = [len(script.text()) for script in scripts.items()]
        l = len(scripts)
        if l > 0:
            return sum(scripts) / l
        else:
            return 0

    def average_script_entropy(self):
        scripts = self.pq('script')
        scripts = [self.__get_entropy(script.text()) for script in scripts.items()]
        l = len(scripts)
        if l > 0:
            return sum(scripts) / l
        else:
            return 0

    def number_of_suspicious_functions(self):
        script_content = self.pq('script').text()
        susf = [1 if i in script_content else 0 for i in self.suspicious_functions]
        return sum(susf)

    def run(self):
        if self.host not in cache:
            try:
                if self.html and self.pq:
                    data = {}
                    data['host'] = self.host
                    data['page_entropy'] = self.url_page_entropy()
                    data['num_script_tags'] = self.number_of_script_tags()
                    data['script_to_body_ratio'] = self.script_to_body_ratio()
                    data['html_length'] = self.length_of_html()
                    data['page_tokens'] = self.number_of_page_tokens()
                    data['num_sentences'] = self.number_of_sentences()
                    data['num_punctuations'] = self.number_of_punctuations()
                    data['distinct_tokens'] = self.number_of_distinct_tokens()
                    data['capitalizations'] = self.number_of_capitalizations()
                    data['avg_tokens_per_sentence'] = self.average_number_of_tokens_in_sentence()
                    data['num_html_tags'] = self.number_of_html_tags()
                    data['num_hidden_tags'] = self.number_of_hidden_tags()
                    data['num_iframes'] = self.number_iframes()
                    data['num_embeds'] = self.number_embeds()
                    data['num_objects'] = self.number_objects()
                    data['hyperlinks'] = self.number_of_hyperlinks()
                    data['num_whitespaces'] = self.number_of_whitespace()
                    data['num_included_elemets'] = self.number_of_included_elements()
                    data['num_double_documents'] = self.number_of_double_documents()
                    data['num_suspicious_elements'] = self.number_of_suspicious_elements()
                    data['num_eval_functions'] = self.number_of_eval_functions()
                    data['avg_script_length'] = self.average_script_length()
                    data['avg_script_entropy'] = self.average_script_entropy()
                    data['num_suspicious_functions'] = self.number_of_suspicious_functions()
                    cache.append(self.host)
                    return data
                else:
                    pass
            except:
                print('OOPS ERROR')
        else:
            print('seen host')
#ob = ContentFeatures("http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/")
#ob.run()
if __name__ == '__main__':
    # files = os.listdir('data')
    files = ['Benign.csv', 'Malware.csv']
    for file in files:
        tag, file = file.split('.')[0], 'FinalDataset/URL/{}'.format(file)
        with open(file, 'r+') as ff:
            lines = [i.strip() for i in ff.readlines()][::-1]
            for line in lines:
                ft = ContentFeatures(line).run()
                if ft is not None:
                    ft['tag'] = tag
                    with open('FinalDataset/output/contentfeatures.json', 'a') as jj:
                        dump(ft, jj)
                        jj.write('\n')
                        print(ft)
                else:
                    pass

Caching 23.39.18.230
Caching 104.244.42.129
Caching 13.224.2.101
Caching 104.244.42.193
Caching 104.244.42.65
Caching 104.244.42.1
Caching 16.16.29.236
Caching 3.13.41.184
Caching 157.240.3.174
Caching 13.48.197.164
Caching 192.0.66.32
Caching 13.51.179.49
Caching 52.14.68.192
Caching 3.13.178.91
Caching 151.101.0.84
Caching 36.52.207.166
Caching 172.67.69.83
Caching 18.65.226.12
Caching 104.244.99.210
Caching 151.101.2.132
Caching 52.193.46.47
Caching 151.101.64.84
Caching 200.201.166.253
Caching 211.233.74.23
Caching 104.247.81.11
Caching 184.28.96.95
Caching 13.224.2.100
Caching 192.0.66.176
Caching 104.244.99.209
Caching 104.26.9.146
Caching 18.155.181.117
Caching 192.0.66.72
Caching 104.26.8.146
Caching 103.224.182.238
Caching 3.113.240.185
Caching 52.84.162.17
Caching 203.171.210.25
Caching 23.193.155.135
Caching 18.172.170.58
Caching 104.244.99.212
Caching 45.60.52.99
Caching 99.84.191.41
Caching 65.8.158.93
Caching 135.84.52.6
Caching 192.0.66.216
Caching 13.224.2.109
Caching 1

#Host-Based Features

In [11]:
!pip install waybackpy
!pip install shodan
!pip install python-whois

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from whois import whois
#from waybackpy import Cdx
from waybackpy import WaybackMachineCDXServerAPI
from socket import gethostbyname
from shodan import Shodan
from requests import get
from urllib.parse import urlparse
from datetime import datetime
from re import compile
from json import dump, loads
from time import sleep

cache = []

with open('FinalDataset/output/hostfeatures.json', 'r+') as ff:
   data = [loads(i) for i in ff.readlines()]
   seen = [i['host'] for i in data]
   for i in seen:
       print('Caching {}'.format(i))
       cache.append(i)

class HostFeatures:
    def __init__(self, url):
        self.url = url
        self.urlparse = urlparse(self.url)
        self.host = self.__get_ip()
        self.now = datetime.now()
        self.init_sub_params = self.initialise_sub_parameters()

    def initialise_sub_parameters(self):
        if self.host not in cache:
            self.whois = self.__get__whois_dict()
            self.shodan = self.__get_shodan_dict()
            self.snapshots = self.__get_site_snapshots()
            return True
        else:
            return False

    def __get_ip(self):
        try:
            ip = self.urlparse.netloc if self.url_host_is_ip() else gethostbyname(self.urlparse.netloc)
            return ip
        except:
            return None

    def __get__whois_dict(self):
        try:
            whois_dict = whois(self.host)
            return whois_dict
        except:
            return {}

    def __get_shodan_dict(self):
        api = Shodan('5Pna4IuOLYCD5i8fSi70zpTiA8bfhU9m')
        try:
            host = api.host(self.host)
            return host
        except:
            return {}

    def __parse__before__date(self, date_string):
        month_year = date_string.split()[-1]
        d = '01-{}'.format(month_year)
        d = datetime.strptime(d, '%d-%b-%Y')
        return d

    def __parse_whois_date(self, date_key):
        cdate = self.whois.get(date_key, None)
        if cdate:
            if isinstance(cdate, str) and 'before' in cdate:
                d = self.__parse__before__date(cdate)
            elif isinstance(cdate, list):
                d = cdate[0]
            else:
                d = cdate
        return d if cdate else cdate

    def __get_site_snapshots(self):
        try:
            #snapshots = Cdx(self.urlparse.netloc).snapshots()
            snapshots = WaybackMachineCDXServerAPI(self.urlparse.netloc).snapshots()
            print('nnnnnn',snapshots.datetime_timestamp)
            snapshotss = [snapshot.datetime_timestamp for snapshot in snapshots]
            return snapshotss
        except:
            return []

    def url_host_is_ip(self):
        host = self.urlparse.netloc
        pattern = compile("^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$")
        match = pattern.match(host)
        return match is not None

    def number_of_subdomains(self):
        ln1 = self.whois.get('nets', None)
        ln2 = self.shodan.get('domains', None)
        ln = ln1 or ln2
        return len(ln) if ln else None

    def url_creation_date(self):
        d = self.__parse_whois_date('creation_date')
        return d

    def url_expiration_date(self):
        d = self.__parse_whois_date('expiration_date')
        return d

    def url_last_updated(self):
        d = self.__parse_whois_date('updated_date')
        return d

    def url_age(self):
        try:
            days = (self.now - self.url_creation_date()).days
        except:
            days = None
        return days

    def url_intended_life_span(self):
        try:
            lifespan = (self.url_expiration_date() - self.url_creation_date()).days
        except:
            lifespan = None
        return lifespan

    def url_life_remaining(self):
        try:
            rem = (self.url_expiration_date() - self.now).days
        except:
            rem = None
        return rem

    def url_registrar(self):
        return self.whois.get('registrar', None)

    def url_registration_country(self):
        c = self.whois.get('country', None)
        return c

    def url_host_country(self):
        c = self.shodan.get('country_name', None)
        return c

    def url_open_ports(self):
        ports = self.shodan.get('ports', '')
        return ports if ports != '' else None

    def url_num_open_ports(self):
        ports = self.url_open_ports()
        lp = len(ports) if ports else 0
        return lp

    def url_is_live(self):
        url = '{}://{}'.format(self.urlparse.scheme, self.urlparse.netloc)
        try:
            return get(url,timeout=5).status_code == 200
        except:
            return False

    def url_isp(self):
        return self.shodan.get('isp', '')

    def url_connection_speed(self):
        url = '{}://{}'.format(self.urlparse.scheme, self.urlparse.netloc)
        if self.url_is_live():
            return get(url).elapsed.total_seconds()
        else:
            return None

    def first_seen(self):
        try:
            fs = self.snapshots[0]
            return fs
        except:
            return datetime.now()

    def get_os(self):
        oss = self.shodan.get('os', None)
        return oss

    def last_seen(self):
        try:
            ls = self.snapshots[-1]
            return ls
        except:
            return datetime.now()

    def days_since_last_seen(self):
        dsls = (self.now - self.last_seen()).days
        return dsls

    def days_since_first_seen(self):
        dsfs = (self.now - self.first_seen()).days
        return dsfs

    def average_update_frequency(self):
        snapshots = self.snapshots
        diffs = [(t-s).days for s, t in zip(snapshots, snapshots[1:])]
        l = len(diffs)
        if l > 0:
            return sum(diffs)/l
        else:
            return 0

    def number_of_updates(self):
        return len(self.snapshots)

    def ttl_from_registration(self):
        earliest_date_seen = self.first_seen()
        try:
            ttl_from_reg = (earliest_date_seen - self.url_creation_date()).days
        except:
            ttl_from_reg = None
        return ttl_from_reg

    def run(self):
        if self.init_sub_params:
            try:
                fv = {
                    "host": self.host,
                    "num_subdomains": self.number_of_subdomains(),
                    "registration_date": str(self.url_creation_date()),
                    "expiration_date": str(self.url_expiration_date()),
                    "last_updates_dates": str(self.url_last_updated()),
                    "age": self.url_age(),
                    "intended_life_span": self.url_intended_life_span(),
                    "life_remaining": self.url_life_remaining(),
                    "registrar": self.url_registrar(),
                    "reg_country": self.url_registration_country(),
                    "host_country": self.url_host_country(),
                    "open_ports": self.url_open_ports(),
                    "num_open_ports": self.url_num_open_ports(),
                    "is_live": self.url_is_live(),
                    "isp": self.url_isp(),
                    "connection_speed": self.url_connection_speed(),
                    "first_seen": str(self.first_seen()),
                    "last_seen": str(self.last_seen()),
                    "days_since_last_seen": self.days_since_last_seen(),
                    "days_since_first_seen": self.days_since_first_seen(),
                    "avg_update_days": self.average_update_frequency(),
                    "total_updates": self.number_of_updates(),
                    "ttl": self.ttl_from_registration()
                }
                cache.append(self.host)
                return fv
            except:
                print('OOPS')
                pass
        else:
            print('Seen Host')
            return None

#ob = HostFeatures("http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/")
#ob.run()

if __name__ == '__main__':
    # files = os.listdir('data')
    files = ['Benign.csv', 'Malware.csv']
    for file in files:
        tag, file = file.split('.')[0], 'FinalDataset/URL/{}'.format(file)
        with open(file, 'r+') as ff:
            lines = [i.strip() for i in ff.readlines()][::-1]
            for line in lines:
                ft = HostFeatures(line).run()
                if ft is not None:
                    ft['tag'] = tag
                    with open('FinalDataset/output/hostfeatures.json', 'a') as jj:
                        dump(ft, jj)
                        jj.write('\n')
                        print(ft)
                else:
                    pass

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
Seen Host
See

#Convert Lexicalfeatures.Json to CSV

In [ ]:
import json
import csv

#lexicalfeatures,contentfeatures,hostfeatures
with open('FinalDataset/output/lexicalfeatures.json') as f:
    jsondata = [json.loads(line) for line in f]
 
data_file = open('FinalDataset/output/lexicalfeatures.csv', 'w', newline='')
csv_writer = csv.writer(data_file)
 
count = 0
for data in jsondata:
    if count == 0:
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data.values())
 
data_file.close()

In [ ]:
import pandas as pd
header = ['host','tld','scheme','url_length','path_length','host_length','host_is_ip','has_port_in_string','num_digits','parameters','fragments','is_encoded','string_entropy','alexa_dis_similarity','subdirectories','periods','has_client','has_login','has_admin','has_server','num_encoded_chars','tag']
data_lexicalfeatures = pd.read_csv('/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/output/lexicalfeatures.csv', names=header)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#xóa dòng đầu
data_lexicalfeatures = data_lexicalfeatures.iloc[1:]

In [ ]:
#lưu lại vào file csv
data_lexicalfeatures.to_csv("/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/output/lexicalfeatures.csv")
data_lexicalfeatures

,host,tld,scheme,url_length,path_length,host_length,host_is_ip,has_port_in_string,num_digits,parameters,...,string_entropy,alexa_dis_similarity,subdirectories,periods,has_client,has_login,has_admin,has_server,num_encoded_chars,tag
1,151.101.193.187,com,https,2081,58,9,False,False,354,40,...,4.731272709910511,0,9,4,False,False,False,False,198,Benign
2,23.39.18.230,com,https,1696,12,12,False,False,1204,2,...,3.0035054057451784,0,2,18,False,False,False,False,76,Benign
3,23.39.18.230,com,https,1517,12,12,False,False,1054,2,...,3.077259199832769,0,2,17,False,False,False,False,71,Benign
4,23.39.18.230,com,https,1348,12,12,False,False,914,2,...,3.1569216803482587,0,2,16,False,False,False,False,66,Benign
5,23.39.18.230,com,https,1189,12,12,False,False,784,2,...,3.243176027699467,0,2,15,False,False,False,False,61,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46940,NaN,cn,http,268,250,11,False,False,83,0,...,3.825334,0,5,3,False,False,True,False,72,Malware
46941,NaN,com,http,378,28,23,False,False,106,7,...,4.716048,0,4,5,True,False,False,False,42,Malware
46942,NaN,com,http,378,28,23,False,False,106,7,...,4.716048,0,4,5,True,False,False,False,42,Malware
46943,NaN,com,http,378,28,23,False,False,106,7,...,4.716048,0,4,5,True,False,False,False,42,Malware


#Convert contentfeatures.json to CSV

In [ ]:
import json
import csv

#lexicalfeatures,contentfeatures,hostfeatures
with open('FinalDataset/output/contentfeatures.json') as f:
    jsondata = [json.loads(line) for line in f]
 
data_file = open('FinalDataset/output/contentfeatures.csv', 'w', newline='')
csv_writer = csv.writer(data_file)
 
count = 0
for data in jsondata:
    if count == 0:
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data.values())
 
data_file.close()

In [ ]:
import pandas as pd
header = ['host',
'page_entropy',
'num_script_tags',
'script_to_body_ratio',
'html_length',
'page_tokens',
'num_sentences',
'num_punctuations',
'distinct_tokens',
'capitalizations',
'avg_tokens_per_sentence',
'num_html_tags',
'num_hidden_tags',
'num_iframes',
'num_embeds',
'num_objects',
'hyperlinks',
'num_whitespaces',
'num_included_elemets',
'num_double_documents',
'num_suspicious_elements',
'num_eval_functions',
'avg_script_length',
'avg_script_entropy',
'num_suspicious_functions',
'tag']
data_contentfeatures = pd.read_csv('/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/output/contentfeatures.csv', names=header)

In [ ]:
#xóa dòng đầu
data_contentfeatures = data_contentfeatures.iloc[1:]

In [ ]:
#lưu lại vào file csv
data_contentfeatures.to_csv("/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/output/contentfeatures.csv")
data_contentfeatures

#Convert hostfeatures.json to CSV

In [ ]:
import json
import csv

#lexicalfeatures,contentfeatures,hostfeatures
with open('FinalDataset/output/hostfeatures.json') as f:
    jsondata = [json.loads(line) for line in f]
 
data_file = open('FinalDataset/output/hostfeatures.csv', 'w', newline='')
csv_writer = csv.writer(data_file)
 
count = 0
for data in jsondata:
    if count == 0:
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data.values())
 
data_file.close()

In [ ]:
import pandas as pd
header = ["host",
"num_subdomains",
"registration_date",
"expiration_date",
"last_updates_dates",
"age",
"intended_life_span",
"life_remaining",
"registrar",
"reg_country",
"host_country",
"open_ports",
"num_open_ports",
"is_live",
"isp",
"connection_speed",
"first_seen",
"last_seen",
"days_since_last_seen",
"days_since_first_seen",
"avg_update_days",
"total_updates",
"ttl",
"tag"]
data_hostfeatures = pd.read_csv('/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/output/hostfeatures.csv', names=header)

In [ ]:
#xóa dòng đầu
data_hostfeatures = data_hostfeatures.iloc[1:]

In [ ]:
#lưu lại vào file csv
data_hostfeatures.to_csv("/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/output/hostfeatures.csv")
data_hostfeatures

#Merge 2 file CSV

In [ ]:
#Tham khảo https://www.geeksforgeeks.org/how-to-merge-two-csv-files-by-specific-column-using-pandas-in-python/
import pandas as pd
  
# reading csv files
data1 = pd.read_csv('/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/output/lexicalfeatures.csv')
data2 = pd.read_csv('/content/drive/MyDrive/NCKH_Phishing attack/Extracting Features URL strings/FinalDataset/output/contentfeatures.csv')
  
# using merge function by setting how='left'
output2 = pd.merge(data1, data2, 
                   on='host', 
                   how='left')
  
# displaying result
output2

In [ ]:
# xoá các giá trị NaN trong host
output2 = output2.dropna(subset = ['host'])

In [ ]:
# xóa cột
utput2 = output2.drop(columns=['Unnamed: 0_x','tag_y'])